In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
from models.mida import Mida
from models.mice import Mice

import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from fancyimpute import KNN, SoftImpute

from preprocessing.data_loading import DataSet
from preprocessing.missing_value_generation import mcar_generator, mar_generator, mnar_generator
from models.imputation_wrapper import SingleImputationWrapper, MultiImputationWrapper
from analysis.evaluation import OutlierEvaluation
from analysis.method_wrapper import OutlierMethodWrapper

In [39]:
dataset = DataSet('ctg', 'outlier')
amputation = mnar_generator(2)
dataset.init_missing_data(amputation)

In [40]:
out_evaluation = OutlierEvaluation(OutlierMethodWrapper(LocalOutlierFactor()), dataset)

In [41]:
# result on complete dataset
out_evaluation.evaluate_result([dataset.complete_data()], 0, 'None', 'None')

In [42]:
imputations = [SingleImputationWrapper(KNN(3)), SingleImputationWrapper(SoftImpute()),
               MultiImputationWrapper(Mice(40))]

In [43]:
#increase missing data rate
for p in [0.2, 0.4, 0.6]:
    dataset.ampute_values(p)
    
    # apply each imputation
    for imputation in imputations:
        results = imputation.complete(dataset.missing_data())
        out_evaluation.evaluate_result(results, p, amputation.name(), imputation.name())

/home/marcus/Studium/Master/Semester_1/Smart Representation/Imputation/preprocessing/missing_value_generation.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Imputing row 1/1831 with 2 missing, elapsed time: 0.962
Imputing row 101/1831 with 0 missing, elapsed time: 0.967
Imputing row 201/1831 with 1 missing, elapsed time: 0.970
Imputing row 301/1831 with 0 missing, elapsed time: 0.973
Imputing row 401/1831 with 2 missing, elapsed time: 0.977
Imputing row 501/1831 with 2 missing, elapsed time: 0.981
Imputing row 601/1831 with 0 missing, elapsed time: 0.984
Imputing row 701/1831 with 1 missing, elapsed time: 0.985
Imputing row 801/1831 with 0 missing, elapsed time: 0.987
Imputing row 901/1831 with 0 missing, elapsed time: 0.988
Imputing row 1001/1831 with 0 missing, elapsed time: 0.989
Imputing row 1101/1831 with 1 missing, elapsed time: 0.992
Imputing row 1201/1831 with 1 missing, elapsed time: 0.994
Imputing row 1301/1831 with 1 missing, elapsed time: 0.997
Imputing row 1401/1831 with 1 missing, elapsed time: 1.000
Imputing row 1501/1831 with 1 missing, elapsed time: 1.004
Imputing row 1601/1831 with 1 missing, elapsed time: 1.007
Imputing 

/home/marcus/anaconda3/lib/python3.6/site-packages/fancyimpute-0.3.1-py3.6.egg/fancyimpute/soft_impute.py:100: RuntimeWarning: divide by zero encountered in double_scalars


[MICE] Completing matrix with shape (1831, 27)
[MICE] Starting imputation round 1/11, elapsed time 0.001
[MICE] Starting imputation round 2/11, elapsed time 0.006
[MICE] Starting imputation round 3/11, elapsed time 0.012
[MICE] Starting imputation round 4/11, elapsed time 0.017
[MICE] Starting imputation round 5/11, elapsed time 0.023
[MICE] Starting imputation round 6/11, elapsed time 0.028
[MICE] Starting imputation round 7/11, elapsed time 0.034
[MICE] Starting imputation round 8/11, elapsed time 0.039
[MICE] Starting imputation round 9/11, elapsed time 0.045
[MICE] Starting imputation round 10/11, elapsed time 0.050
[MICE] Starting imputation round 11/11, elapsed time 0.055
[MICE] Completing matrix with shape (1831, 27)
[MICE] Starting imputation round 1/11, elapsed time 0.001
[MICE] Starting imputation round 2/11, elapsed time 0.006
[MICE] Starting imputation round 3/11, elapsed time 0.012
[MICE] Starting imputation round 4/11, elapsed time 0.017
[MICE] Starting imputation round 5

[MICE] Starting imputation round 7/11, elapsed time 0.047
[MICE] Starting imputation round 8/11, elapsed time 0.054
[MICE] Starting imputation round 9/11, elapsed time 0.061
[MICE] Starting imputation round 10/11, elapsed time 0.067
[MICE] Starting imputation round 11/11, elapsed time 0.075
[MICE] Completing matrix with shape (1831, 27)
[MICE] Starting imputation round 1/11, elapsed time 0.001
[MICE] Starting imputation round 2/11, elapsed time 0.007
[MICE] Starting imputation round 3/11, elapsed time 0.012
[MICE] Starting imputation round 4/11, elapsed time 0.018
[MICE] Starting imputation round 5/11, elapsed time 0.024
[MICE] Starting imputation round 6/11, elapsed time 0.029
[MICE] Starting imputation round 7/11, elapsed time 0.035
[MICE] Starting imputation round 8/11, elapsed time 0.041
[MICE] Starting imputation round 9/11, elapsed time 0.046
[MICE] Starting imputation round 10/11, elapsed time 0.052
[MICE] Starting imputation round 11/11, elapsed time 0.058
[MICE] Completing mat

...

[MICE] Starting imputation round 2/11, elapsed time 0.012
[MICE] Starting imputation round 3/11, elapsed time 0.029
[MICE] Starting imputation round 4/11, elapsed time 0.038
[MICE] Starting imputation round 5/11, elapsed time 0.044
[MICE] Starting imputation round 6/11, elapsed time 0.050
[MICE] Starting imputation round 7/11, elapsed time 0.057
[MICE] Starting imputation round 8/11, elapsed time 0.063
[MICE] Starting imputation round 9/11, elapsed time 0.069
[MICE] Starting imputation round 10/11, elapsed time 0.075
[MICE] Starting imputation round 11/11, elapsed time 0.080
[MICE] Completing matrix with shape (1831, 27)
[MICE] Starting imputation round 1/11, elapsed time 0.001
[MICE] Starting imputation round 2/11, elapsed time 0.007
[MICE] Starting imputation round 3/11, elapsed time 0.013
[MICE] Starting imputation round 4/11, elapsed time 0.018
[MICE] Starting imputation round 5/11, elapsed time 0.024
[MICE] Starting imputation round 6/11, elapsed time 0.030
[MICE] Starting imputat

[MICE] Starting imputation round 11/11, elapsed time 0.079
[MICE] Completing matrix with shape (1831, 27)
[MICE] Starting imputation round 1/11, elapsed time 0.001
[MICE] Starting imputation round 2/11, elapsed time 0.008
[MICE] Starting imputation round 3/11, elapsed time 0.014
[MICE] Starting imputation round 4/11, elapsed time 0.020
[MICE] Starting imputation round 5/11, elapsed time 0.026
[MICE] Starting imputation round 6/11, elapsed time 0.032
[MICE] Starting imputation round 7/11, elapsed time 0.038
[MICE] Starting imputation round 8/11, elapsed time 0.044
[MICE] Starting imputation round 9/11, elapsed time 0.049
[MICE] Starting imputation round 10/11, elapsed time 0.055
[MICE] Starting imputation round 11/11, elapsed time 0.061
[MICE] Completing matrix with shape (1831, 27)
[MICE] Starting imputation round 1/11, elapsed time 0.001
[MICE] Starting imputation round 2/11, elapsed time 0.007
[MICE] Starting imputation round 3/11, elapsed time 0.013
[MICE] Starting imputation round 

[MICE] Completing matrix with shape (1831, 27)
[MICE] Starting imputation round 1/11, elapsed time 0.001
[MICE] Starting imputation round 2/11, elapsed time 0.007
[MICE] Starting imputation round 3/11, elapsed time 0.013
[MICE] Starting imputation round 4/11, elapsed time 0.019
[MICE] Starting imputation round 5/11, elapsed time 0.025
[MICE] Starting imputation round 6/11, elapsed time 0.031
[MICE] Starting imputation round 7/11, elapsed time 0.039
[MICE] Starting imputation round 8/11, elapsed time 0.045
[MICE] Starting imputation round 9/11, elapsed time 0.051
[MICE] Starting imputation round 10/11, elapsed time 0.057
[MICE] Starting imputation round 11/11, elapsed time 0.063
[MICE] Completing matrix with shape (1831, 27)
[MICE] Starting imputation round 1/11, elapsed time 0.001
[MICE] Starting imputation round 2/11, elapsed time 0.007
[MICE] Starting imputation round 3/11, elapsed time 0.013
[MICE] Starting imputation round 4/11, elapsed time 0.019
[MICE] Starting imputation round 5

[MICE] Starting imputation round 2/11, elapsed time 0.026
[MICE] Starting imputation round 3/11, elapsed time 0.032
[MICE] Starting imputation round 4/11, elapsed time 0.038
[MICE] Starting imputation round 5/11, elapsed time 0.046
[MICE] Starting imputation round 6/11, elapsed time 0.052
[MICE] Starting imputation round 7/11, elapsed time 0.058
[MICE] Starting imputation round 8/11, elapsed time 0.064
[MICE] Starting imputation round 9/11, elapsed time 0.070
[MICE] Starting imputation round 10/11, elapsed time 0.076
[MICE] Starting imputation round 11/11, elapsed time 0.082
[MICE] Completing matrix with shape (1831, 27)
[MICE] Starting imputation round 1/11, elapsed time 0.001
[MICE] Starting imputation round 2/11, elapsed time 0.007
[MICE] Starting imputation round 3/11, elapsed time 0.013
[MICE] Starting imputation round 4/11, elapsed time 0.019
[MICE] Starting imputation round 5/11, elapsed time 0.025
[MICE] Starting imputation round 6/11, elapsed time 0.031
[MICE] Starting imputat

[MICE] Starting imputation round 2/11, elapsed time 0.027
[MICE] Starting imputation round 3/11, elapsed time 0.033
[MICE] Starting imputation round 4/11, elapsed time 0.039
[MICE] Starting imputation round 5/11, elapsed time 0.047
[MICE] Starting imputation round 6/11, elapsed time 0.055
[MICE] Starting imputation round 7/11, elapsed time 0.061
[MICE] Starting imputation round 8/11, elapsed time 0.067
[MICE] Starting imputation round 9/11, elapsed time 0.073
[MICE] Starting imputation round 10/11, elapsed time 0.079
[MICE] Starting imputation round 11/11, elapsed time 0.085
[MICE] Completing matrix with shape (1831, 27)
[MICE] Starting imputation round 1/11, elapsed time 0.001
[MICE] Starting imputation round 2/11, elapsed time 0.007
[MICE] Starting imputation round 3/11, elapsed time 0.013
[MICE] Starting imputation round 4/11, elapsed time 0.019
[MICE] Starting imputation round 5/11, elapsed time 0.025
[MICE] Starting imputation round 6/11, elapsed time 0.032
[MICE] Starting imputat

In [44]:
out_evaluation.evaluation_results

,"(settings, p)","(settings, mechanism)","(settings, imputation)","(settings, number)","(auc, post)","(auc, prev)"
0,0,None,None,1,0.367310,0.367310
1,0.2,mnar,KNN,1,0.373088,0.373088
2,0.2,mnar,SoftImpute,1,0.368793,0.368793
3,0.2,mnar,Mice,1,0.369668,0.369668
4,0.2,mnar,Mice,5,0.383827,0.369843
5,0.2,mnar,Mice,10,0.389944,0.369006
6,0.2,mnar,Mice,15,0.394579,0.368783
7,0.2,mnar,Mice,20,0.397284,0.368638
8,0.2,mnar,Mice,25,0.399509,0.368752
9,0.2,mnar,Mice,30,0.400731,0.369188
